<a href="https://colab.research.google.com/github/JoseRAdan/information_retrieval_eq2/blob/main/Avance3_Equipo2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AVANCE 3 EQUIPO 2
## PRYECTO INTEGRADOR


## ARQUITECTURA

Basado en la arquitectura genérica RETRIEVER-READER.

Usando la herramienta **Haystack** para validación del modelo de lenguage de **Hugging Face Hub** llamado "**dccuchile/albert-base-spanish-finetuned-qa-mlqa**" en su aplicabilidad de extraer respuestas acertadas del tipo de información que representa la normatividad académica.

## NOTAS PARA EJECUCION EN COLAB

Aunque corre sin GPU, es recomendable activarlo si esta disponible.

## Instalar Haystack

In [2]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,inference]

  Using cached farm_haystack-1.25.5-py3-none-any.whl.metadata (32 kB)
  Using cached transformers-4.39.3-py3-none-any.whl.metadata (134 kB)
  Using cached sentence_transformers-2.7.0-py3-none-any.whl.metadata (11 kB)
  Using cached accelerate-0.30.1-py3-none-any.whl.metadata (18 kB)
Using cached transformers-4.39.3-py3-none-any.whl (8.8 MB)
Using cached sentence_transformers-2.7.0-py3-none-any.whl (171 kB)
Using cached farm_haystack-1.25.5-py3-none-any.whl (770 kB)
Using cached accelerate-0.30.1-py3-none-any.whl (302 kB)


## Puede ser necesario reiniciar después de instalar el siguiente

In [3]:
!pip install farm-haystack[elasticsearch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 671.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.0/386.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


Set the logging level to INFO:

In [5]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

## Preparar fuente de datos

####Copiar PDFs en data/pdf
####Crear data/txt
####Crear data/txt_norm para guardar archivos con texto normalizado (pruebas)
####Usar libreria PyPDF2 para extraer texto de archivos

In [4]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 1.4 MB/s eta 0:00:00


Normalizar datos usando NTLK

In [6]:
!pip install nltk
import nltk
import string
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
def clean_text(texto_extraido):
  texto_limpio = nltk.corpus.stopwords.words("spanish") + ["¿", "¡", ".", ",", ":", ";", "!", "?", "\"", "\'", "\n", "\t"]
#  texto_sin_stopwords = [palabra for palabra in texto_extraido.split() if palabra not in texto_limpio]
  texto_sin_stopwords = "".join([palabra + ' ' for palabra in texto_extraido.split() if palabra not in texto_limpio])
#  texto_sin_puntuacion = "".join([caracter if caracter not in string.punctuation else " " for caracter in texto_sin_stopwords])
  texto_normalizado = texto_sin_stopwords.lower()#texto_sin_puntuacion.lower()
  return texto_normalizado

## Extrayendo text de PDF y guardarlos como archivos txt


In [8]:
import os
import PyPDF2

# Carpeta donde se encuentran los PDFs
carpeta_pdf = "/content/data/pdf"
carpeta_txt = "/content/data/txt"

# Recorre cada archivo PDF en la carpeta
for archivo in os.listdir(carpeta_pdf):
    if archivo.endswith(".pdf"):
        # Extrae el nombre del archivo sin la extensión
        nombre_archivo, _ = os.path.splitext(archivo)

        # Abre el archivo PDF en modo lectura binaria
        with open(os.path.join(carpeta_pdf, archivo), 'rb') as pdf_file:
            # Crea un lector de PDF
            lector = PyPDF2.PdfReader(pdf_file)

            # Extrae el texto de cada página
            texto = ""
            for pagina in lector.pages:
                # Extrae el texto de la página
                texto_pagina = pagina.extract_text()
                texto += texto_pagina

            texto_normalizado = clean_text(texto)

            # Guarda el texto en un archivo de texto con el nombre original
            with open(carpeta_txt + '/' + f"{nombre_archivo}.txt", "w", encoding="utf-8") as archivo_txt:
                archivo_txt.write(texto)
            # Guarda el texto normalizado en un archivo de texto con el nombre original
            with open(carpeta_txt + '_norm/' + f"{nombre_archivo}.txt", "w", encoding="utf-8") as archivo_txt:
                archivo_txt.write(texto_normalizado)

## Inicializar DocumentStore

We'll start creating our question answering system by initializing a DocumentStore. A DocumentStore stores the Documents that the question answering system uses to find answers to your questions. In this tutorial, we're using the `InMemoryDocumentStore`, which is the simplest DocumentStore to get started with. It requires no external dependencies and it's a good option for smaller projects and debugging. But it doesn't scale up so well to larger Document collections, so it's not a good choice for production systems. To learn more about the DocumentStore and the different types of external databases that we support, see [DocumentStore](https://docs.haystack.deepset.ai/docs/document_store).

Let's initialize the the DocumentStore:

In [9]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


##Indexar informacion de archivos txt

In [10]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [carpeta_txt + "/" + f for f in os.listdir(carpeta_txt)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.
Preprocessing: 100%|██████████| 184/184 [00:03<00:00, 53.91docs/s]
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '7269be1e35cc0bcb9465a0455a655977' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'd9cf73971066b31f054caed5b5fd790a' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '2371dd5c515c8437b3ba540c50008c17' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '5226f77aa14c0613e089dd8a0357e7fd' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'b0e220f7b996835437b3351b3c7fb8c0' already exists in inde

{'documents': [<Document: {'content': 'Disposiciones Académicas para\nProgramas de Internacionalización de\nAlumnos del Tec de Monterrey\nVersión 1.0\nVicerrectoría de Internacionalización\nNoviembre, 2002Disposiciones Académicas para Programas  de Internacionalización\nSistema ITESM\n07 de noviembre de 2002\n1ÍNDICE\ni. INTRODUCCIÓN ........................................................................................................... 2\nii. ALCANCES DE ESTE DOCUMENTO ........................................................................ 2\n1. CONVENIOS Y ACUERDOS DE COOPERACIÓN ACADÉMICA\nINTERNACIONAL ............................................................................................................ 3\n1.1 De los criterios de afiliación del Tec de Monterrey .............................................. 3\n1.2 De la administración de convenios ....................................................................... 4\n1.3 De la generación y administración de programas 

## Initializar el Retriever

Our search system will use a Retriever, so we need to initialize it. A Retriever sifts through all the Documents and returns only the ones relevant to the question. This tutorial uses the BM25 algorithm. For more Retriever options, see [Retriever](https://docs.haystack.deepset.ai/docs/retriever).

Let's initialize a BM25Retriever and make it use the InMemoryDocumentStore we initialized earlier in this tutorial:

In [11]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

The Retriever is ready but we still need to initialize the Reader.

## Inicializar el Reader

A Reader scans the texts it received from the Retriever and extracts the top answer candidates. Readers are based on powerful deep learning models but are much slower than Retrievers at processing the same amount of text. In this tutorial, we're using a FARMReader with a base-sized RoBERTa question answering model called [`deepset/roberta-base-squad2`](https://huggingface.co/deepset/roberta-base-squad2). It's a strong all-round model that's good as a starting point. To find the best model for your use case, see [Models](https://haystack.deepset.ai/pipeline_nodes/reader#models).

Let's initialize the Reader:

In [12]:
from haystack.nodes import FARMReader

#reader = FARMReader(model_name_or_path="dccuchile/bert-base-spanish-wwm-uncased-finetuned-qa-mlqa", use_gpu=True)
reader = FARMReader(model_name_or_path="dccuchile/albert-base-spanish-finetuned-qa-mlqa", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'dccuchile/albert-base-spanish-finetuned-qa-mlqa' (Albert)


pytorch_model.bin:   0%|          | 0.00/44.9M [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model:Auto-detected model language: spanish
INFO:haystack.modeling.model.language_model:Loaded 'dccuchile/albert-base-spanish-finetuned-qa-mlqa' (Albert model) from model hub.


tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


We've initalized all the components for our pipeline. We're now ready to create the pipeline.

## Creacion del Pilelien Retriever-Reader


In [13]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

The pipeline's ready, you can now go ahead and ask a question!

## Probar haciendo una pregunta

1. Use the pipeline `run()` method to ask a question. The query argument is where you type your question. Additionally, you can set the number of documents you want the Reader and Retriever to return using the `top-k` parameter. To learn more about setting arguments, see [Arguments](https://docs.haystack.deepset.ai/docs/pipelines#arguments). To understand the importance of the `top-k` parameter, see [Choosing the Right top-k Values](https://docs.haystack.deepset.ai/docs/optimization#choosing-the-right-top-k-values).

In [40]:
#prediction = pipe.run(
#    query="Qué es el período sabático?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
#)
prediction = pipe.run(
    query="¿Cuál es la definición exacta de un crédito académico en nuestra institución para los 3 niveles Preparatoria, Profesional y Posgrado?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)
#prediction = pipe.run(
#  query="¿Cuál es la calificación aprobatoria en nivel Posgrado?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
#)
#prediction = pipe.run(
#  query="¿Para cuáles niveles de estudio aplican los Lineamientos de Credenciales Alternativas?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
#)

Inferencing Samples: 100%|██████████| 1/1 [00:09<00:00,  9.75s/ Batches]


In [41]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'Posgrado', 'type': 'extractive', 'score': 0.6020541191101074, 'context': 'valuación y Credenciales Alternativas  y los correspondientes a  nivel Posgrado en la sección\nPlanes de Estudio, de la página de la Vicerrectoría de I', 'offsets_in_document': [{'start': 185, 'end': 193}], 'offsets_in_context': [{'start': 71, 'end': 79}], 'document_ids': ['c92d03793da07f82a59545550215d738'], 'meta': {'_split_id': 1}}>,
             <Answer {'answer': 'Profesional y Posgrado', 'type': 'extractive', 'score': 0.5507513284683228, 'context': 'ro del Tecnológico de Monterrey\nque aplican para los niveles de Profesional y Posgrado, así como para Educación Continua, además de los\nlineamientos p', 'offsets_in_document': [{'start': 224, 'end': 246}], 'offsets_in_context': [{'start': 64, 'end': 86}], 'document_ids': ['cf9b5f701bc6b06662cb7cc72abb41fb'], 'meta': {'_split_id': 0}}>,
             <Answer {'answer': 'Profesional, Posgrado y Educación Continua', 'type': 'extrac

In [42]:
from haystack.utils import print_answers

print_answers(prediction, details="all")  ## Choose from `minimum`, `medium`, and `all`

('Query: ¿Para cuáles niveles de estudio aplican los Lineamientos de '
 'Credenciales Alternativas?')
'Answers:'
[   <Answer {'answer': 'Posgrado', 'type': 'extractive', 'score': 0.6020541191101074, 'context': 'valuación y Credenciales Alternativas  y los correspondientes a  nivel Posgrado en la sección\nPlanes de Estudio, de la página de la Vicerrectoría de I', 'offsets_in_document': [{'start': 185, 'end': 193}], 'offsets_in_context': [{'start': 71, 'end': 79}], 'document_ids': ['c92d03793da07f82a59545550215d738'], 'meta': {'_split_id': 1}}>,
    <Answer {'answer': 'Profesional y Posgrado', 'type': 'extractive', 'score': 0.5507513284683228, 'context': 'ro del Tecnológico de Monterrey\nque aplican para los niveles de Profesional y Posgrado, así como para Educación Continua, además de los\nlineamientos p', 'offsets_in_document': [{'start': 224, 'end': 246}], 'offsets_in_context': [{'start': 64, 'end': 86}], 'document_ids': ['cf9b5f701bc6b06662cb7cc72abb41fb'], 'meta': {'_split_id': 0}}>